In [32]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.preprocessing import LabelEncoder

In [133]:
def load_data():
    columns = ["Col1","Col 2","Col 3","Col 4","Col 5","Col 6", "Col 7", "Col 8","Col 9","Col 10"]
    data = pd.read_csv("/users/Sushanta/Documents/GitHub/Illinois/CS598 Practical Statistical Learning/Project/data/Ames_data.csv")
    testIds = pd.read_csv("/users/Sushanta/Documents/GitHub/Illinois/CS598 Practical Statistical Learning/Project/data/project1_testIDs.dat",sep = " ", names = columns)
    
    return data, testIds

def created_train_test(data,testIds,j):
    j = j-1
    test = data.iloc[np.array(testIds)[:,j]]
    train = data.drop(np.array(testIds)[:,j], axis=0)
    return train,test

def feature_engineering(x_train,x_test):
    #number of record
    train_num = x_train.shape[0]
    test_num = x_test.shape[0]
    #Merge the Train & Test
    df = [x_train,x_test]
    df_train_test = pd.concat(df)
    
    #Label Encoder to transform the Categoricsal Variable
    lbe = LabelEncoder()
    for col_name in train.columns[train.dtypes == 'object']:
        #col_name = cols+'_Cat'
        df_train_test[col_name] = lbe.fit_transform(df_train_test[col_name])
        
    #Let's drop the column "Garage_Yr_Blt" for now, as it ha NaN
    df_train_test = df_train_test.drop(['Garage_Yr_Blt'],axis=1)
    x_train = pd.DataFrame(np.array(df_train_test)[:train_num,:],columns = df_train_test.columns)
    x_test = pd.DataFrame(np.array(df_train_test)[train_num:,:],columns = df_train_test.columns)
    
    return x_train,x_test

def lasso_model(x_train,y_train,x_test,y_test,alpha=0.5):
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(x_train,y_train)
    y_predict = lasso_model.predict(x_test)
    return lasso_model,y_predict
    

In [135]:
data, testIds = load_data()
train,test = created_train_test(data,testIds,j=1)

#Print the dataset size
print ("Number of elements in the Training Set: {}".format(train.shape))
print ("Number of elements in the Test Set: {}".format(test.shape))

#Write to the Train.csv file
train.to_csv("/users/Sushanta/Documents/GitHub/Illinois/CS598 Practical Statistical Learning/Project/data/train.csv",index=False)
test.to_csv("/users/Sushanta/Documents/GitHub/Illinois/CS598 Practical Statistical Learning/Project/data/test.csv",index=False)

y_train = train['Sale_Price']
x_train = train.drop(['Sale_Price'],axis=1)
y_test = test['Sale_Price']
x_test = test.drop(['Sale_Price'],axis=1)

x_train_transformed,x_test_transformed = feature_engineering(x_train,x_test)

#Print the dataset size
print ("Number of elements in the Train Transformed Set: {}".format(x_train_transformed.shape))
print ("Number of elements in the Test Transformed Set: {}".format(x_test_transformed.shape))


Number of elements in the Training Set: (2051, 83)
Number of elements in the Test Set: (879, 83)
Number of elements in the Train Transformed Set: (2051, 81)
Number of elements in the Test Transformed Set: (879, 81)


In [81]:
x_train_transformed[x_train_transformed['MS_SubClass'].isna()]

Empty DataFrame
Columns: [PID, MS_SubClass, MS_Zoning, Lot_Frontage, Lot_Area, Street, Alley, Lot_Shape, Land_Contour, Utilities, Lot_Config, Land_Slope, Neighborhood, Condition_1, Condition_2, Bldg_Type, House_Style, Overall_Qual, Overall_Cond, Year_Built, Year_Remod_Add, Roof_Style, Roof_Matl, Exterior_1st, Exterior_2nd, Mas_Vnr_Type, Mas_Vnr_Area, Exter_Qual, Exter_Cond, Foundation, Bsmt_Qual, Bsmt_Cond, Bsmt_Exposure, BsmtFin_Type_1, BsmtFin_SF_1, BsmtFin_Type_2, BsmtFin_SF_2, Bsmt_Unf_SF, Total_Bsmt_SF, Heating, Heating_QC, Central_Air, Electrical, First_Flr_SF, Second_Flr_SF, Low_Qual_Fin_SF, Gr_Liv_Area, Bsmt_Full_Bath, Bsmt_Half_Bath, Full_Bath, Half_Bath, Bedroom_AbvGr, Kitchen_AbvGr, Kitchen_Qual, TotRms_AbvGrd, Functional, Fireplaces, Fireplace_Qu, Garage_Type, Garage_Yr_Blt, Garage_Finish, Garage_Cars, Garage_Area, Garage_Qual, Garage_Cond, Paved_Drive, Wood_Deck_SF, Open_Porch_SF, Enclosed_Porch, Three_season_porch, Screen_Porch, Pool_Area, Pool_QC, Fence, Misc_Feature, Misc_Val, Mo_Sold, Year_Sold, Sale_Type, Sale_Condition, Longitude, Latitude]
Index: []

[0 rows x 82 columns]

In [136]:
lasso_model,y_predict = lasso_model(x_train_transformed,y_train,x_test_transformed,y_test)

/Users/sushanta/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 978522006459.4314, tolerance: 1342123403.9788368
  positive)


In [143]:
y_predict[0]

130059.4778816551

In [153]:
np.array(y_test)[3]

148000

In [140]:
y_predict.shape

(879,)